In [ ]:
import kagglehub

# Download latest version of recipeNLG
path_recipeNlg = kagglehub.dataset_download("paultimothymooney/recipenlg")

print("Path to dataset files:", path_recipeNlg)

100%|██████████| 643M/643M [00:03<00:00, 219MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/paultimothymooney/recipenlg/versions/1


In [ ]:
# Download latest version of ubereat restaurant menu
path_ubereatMenu = kagglehub.dataset_download("ahmedshahriarsakib/uber-eats-usa-restaurants-menus")

print("Path to dataset files:", path_ubereatMenu)

100%|██████████| 193M/193M [00:01<00:00, 142MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ahmedshahriarsakib/uber-eats-usa-restaurants-menus/versions/12


In [ ]:
import os
import pandas as pd

# List all files in the dataset path
print(os.listdir(path_recipeNlg))

['RecipeNLG_license.png', 'RecipeNLG_code', 'RecipeNLG_dataset.csv', 'RecipeNLG_paper.pdf']


In [ ]:
print(os.listdir(path_ubereatMenu))

['restaurant-menus.csv', 'restaurants.csv']


In [ ]:
# make dataframe of each database
df_recipe = pd.read_csv(os.path.join(path_recipeNlg, "RecipeNLG_dataset.csv"))
df_recipe.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
df_restaurant = pd.read_csv(os.path.join(path_ubereatMenu, "restaurants.csv"))
df_restaurant.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700


In [ ]:
df_restaurant_menu = pd.read_csv(os.path.join(path_ubereatMenu, "restaurant-menus.csv"))
df_restaurant_menu.head()

,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD


In [ ]:
# Make copies of the original dataframes
df_recipe_raw = df_recipe.copy()
df_restaurant_menu_raw = df_restaurant_menu.copy()
df_restaurant_raw = df_restaurant.copy()

In [ ]:
import numpy as np

# Set chunk size (you can tweak this lower if still OOM)
chunk_size = 50_000
num_chunks = int(np.ceil(len(df_recipe_raw) / chunk_size))

# Split df_recipe_raw into list of smaller chunks
recipe_chunks = [df_recipe_raw.iloc[i*chunk_size:(i+1)*chunk_size].copy() for i in range(num_chunks)]

In [ ]:
import ast
import re

# Helper functions
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except:
        return []

def clean_name(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Clean and collect
cleaned_chunks = []

for raw_chunk in recipe_chunks:
    # Make a clean copy first thing to avoid view issues
    chunk = raw_chunk.copy()

    # Drop missing data and re-copy to be extra safe
    chunk = chunk.dropna(subset=['title', 'ingredients', 'directions']).copy()

    # Convert stringified fields
    chunk['ingredients'] = chunk['ingredients'].apply(safe_literal_eval)
    chunk['directions'] = chunk['directions'].apply(safe_literal_eval)
    chunk['NER'] = chunk['NER'].apply(safe_literal_eval)

    # Filter short recipes
    chunk = chunk[chunk['ingredients'].map(len) > 1].copy()
    chunk = chunk[chunk['directions'].map(len) > 1].copy()

    # Clean recipe title
    chunk['clean_title'] = chunk['title'].apply(clean_name)

    # Add cleaned chunk to the list
    cleaned_chunks.append(chunk)

In [ ]:
df_recipe_cleaned = pd.concat(cleaned_chunks, ignore_index=True)
print(f"✅ Cleaned recipe dataset shape: {df_recipe_cleaned.shape}")
df_recipe_cleaned.head()

✅ Cleaned recipe dataset shape: (2050520, 8)


,Unnamed: 0,title,ingredients,directions,link,source,NER,clean_title
0,0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","[In a heavy 2-quart saucepan, mix brown sugar,...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[brown sugar, milk, vanilla, nuts, butter, bit...",nobake nut cookies
1,1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...","[Place chipped beef on bottom of baking dish.,...",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[beef, chicken breasts, cream of mushroom soup...",jewell balls chicken
2,3,Chicken Funny,"[1 large whole chicken, 2 (10 1/2 oz.) cans ch...","[Boil and debone chicken., Put bite size piece...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[chicken, chicken gravy, cream of mushroom sou...",chicken funny
3,4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker cru...",[Combine first four ingredients and press in 1...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[peanut butter, graham cracker crumbs, butter,...",reeses cupscandy
4,5,Cheeseburger Potato Soup,"[6 baking potatoes, 1 lb. of extra lean ground...",[Wash potatoes; prick several times with a for...,www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[baking potatoes, extra lean ground beef, butt...",cheeseburger potato soup


Clean df_restaurant_menu_raw

In [ ]:
import re
import html

df_restaurant_menu_raw.to_csv("df_restaurant_menu_raw.csv", index=False)
# Helper functions
def clean_name(text):
    if pd.isna(text):
        return ""
    text = html.unescape(str(text))
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_price(val):
    if pd.isna(val):
        return None
    match = re.search(r'(\d+\.\d+)', str(val))
    return float(match.group(1)) if match else None

# Clean in chunks
chunk_size = 50_000
cleaned_chunks = []

for chunk in pd.read_csv("df_restaurant_menu_raw.csv", chunksize=chunk_size):
    chunk = chunk.copy()

    # Clean name
    chunk['clean_name'] = chunk['name'].apply(clean_name)

    # Clean price
    chunk['price_clean'] = chunk['price'].apply(extract_price)

    # Clean description
    chunk['description'] = chunk['description'].apply(lambda x: html.unescape(str(x)).strip())

    cleaned_chunks.append(chunk)

# Combine all chunks
df_menu_cleaned = pd.concat(cleaned_chunks, ignore_index=True)

# Save cleaned version
df_menu_cleaned.to_csv("df_menu_cleaned.csv", index=False)

print("✅ df_menu_cleaned saved successfully.")

✅ df_menu_cleaned saved successfully.


In [ ]:
# Save recipe data
df_recipe_cleaned.to_csv("df_recipe_cleaned.csv", index=False)
print("✅ df_recipe_cleaned saved successfully.")

✅ df_recipe_cleaned saved successfully.


In [ ]:
import numpy as np
import random

# Make a clean copy
df_restaurant_cleaned = df_restaurant_raw.copy()

# Normalize column names
df_restaurant_cleaned.columns = [col.strip().lower().replace(" ", "_") for col in df_restaurant_cleaned.columns]

# Clean up strings
for col in ['name', 'address', 'zip_code', 'category', 'price_range']:
    if col in df_restaurant_cleaned.columns:
        df_restaurant_cleaned[col] = df_restaurant_cleaned[col].astype(str).str.strip()

# Fill missing score with random floats between 0 and 5
if 'score' in df_restaurant_cleaned.columns:
    df_restaurant_cleaned['score'] = df_restaurant_cleaned['score'].apply(
        lambda x: round(random.uniform(0, 5), 2) if pd.isna(x) else x
    )

# Fill missing rating with random floats between 1 and 5 (safe default)
if 'rating' in df_restaurant_cleaned.columns:
    df_restaurant_cleaned['rating'] = df_restaurant_cleaned['rating'].apply(
        lambda x: round(random.uniform(1, 5), 2) if pd.isna(x) else x
    )

# Fill missing price_range with random $ level from 1 to 4
if 'price_range' in df_restaurant_cleaned.columns:
    dollar_levels = ['$','$$','$$$','$$$$']
    df_restaurant_cleaned['price_range'] = df_restaurant_cleaned['price_range'].apply(
        lambda x: random.choice(dollar_levels) if pd.isna(x) or x.strip() == '' else x
    )

# Create price_level column for numeric representation
df_restaurant_cleaned['price_level'] = df_restaurant_cleaned['price_range'].apply(lambda x: len(str(x)) if '$' in str(x) else None)

# Clean name for future matching
df_restaurant_cleaned['clean_name'] = df_restaurant_cleaned['name'].str.lower().str.replace(r'[^a-z0-9 ]', '', regex=True).str.strip()

# Drop any remaining restaurants missing coordinates
df_restaurant_cleaned.dropna(subset=['name', 'lat', 'lng'], inplace=True)

# Save cleaned version
df_restaurant_cleaned.to_csv("df_restaurant_cleaned.csv", index=False)

print("✅ Restaurant DataFrame cleaned and saved successfully.")

✅ Restaurant DataFrame cleaned and saved successfully.


In [ ]:
import pandas as pd
df_menu_cleaned=pd.read_csv("df_menu_cleaned.csv")
df_menu_cleaned.head()

,restaurant_id,category,name,description,price,clean_name,price_clean
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD,extra large meat lovers,15.99
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD,extra large supreme,15.99
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD,extra large pepperoni,14.99
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD,extra large bbq chicken bacon,15.99
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD,extra large 5 cheese,14.99


In [ ]:
df_recipe_cleaned=pd.read_csv("df_recipe_cleaned.csv")
df_recipe_cleaned.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,clean_title
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","['In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...",nobake nut cookies
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...",['Place chipped beef on bottom of baking dish....,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"['beef', 'chicken breasts', 'cream of mushroom...",jewell balls chicken
2,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","['Boil and debone chicken.', 'Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"['chicken', 'chicken gravy', 'cream of mushroo...",chicken funny
3,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...",['Combine first four ingredients and press in ...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"['peanut butter', 'graham cracker crumbs', 'bu...",reeses cupscandy
4,5,Cheeseburger Potato Soup,"['6 baking potatoes', '1 lb. of extra lean gro...",['Wash potatoes; prick several times with a fo...,www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"['baking potatoes', 'extra lean ground beef', ...",cheeseburger potato soup


In [ ]:
df_restaurant_cleaned=pd.read_csv("df_restaurant_cleaned.csv")
df_restaurant_cleaned.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng,price_level,clean_name
0,1,19,PJ Fresh (224 Daniel Payne Drive),4.25,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,1.0,pj fresh 224 daniel payne drive
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,1.85,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330,NaN,j tiz smoothiencoffee bar
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),2.97,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640,1.0,philly fresh cheesesteaks 541b graymont ave
3,4,17,Papa Murphy's (1580 Montgomery Highway),4.03,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614,1.0,papa murphys 1580 montgomery highway
4,5,162,Nelson Brothers Cafe (17th St N),4.70,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700,NaN,nelson brothers cafe 17th st n


In [ ]:
import pandas as pd

# Extract and clean unique categories
unique_categories = df_menu_cleaned['category'].dropna().astype(str).str.strip().unique()
unique_categories = sorted(unique_categories)

# Save to CSV
category_df = pd.DataFrame(unique_categories, columns=['original_category'])
category_df.to_csv("menu_unique_categories.csv", index=False)

print("✅ Saved as 'menu_unique_categories.csv'")

✅ Saved as 'menu_unique_categories.csv'


In [ ]:
import pandas as pd
import re
import random

# Load the cleaned restaurant DataFrame
df_restaurant_cleaned = pd.read_csv("df_restaurant_cleaned.csv")

# 1. Create a new column 'clean_name' that removes any parenthetical content from 'name' and converts to lowercase
def clean_name(name):
    return re.sub(r'\([^)]*\)', '', name).strip()

df_restaurant_cleaned['clean_name'] = df_restaurant_cleaned['name'].apply(clean_name)

# 2. Create a new column 'ratings_count_rand' that randomizes the rating count only if the original value is NaN
df_restaurant_cleaned['ratings_count_rand'] = df_restaurant_cleaned['ratings'].apply(
    lambda x: random.randint(10, 500) if pd.isna(x) else x
)

# 3. Create a new column 'price_level_rand' that randomizes the price level only if the original value is NaN
df_restaurant_cleaned['price_level_rand'] = df_restaurant_cleaned['price_level'].apply(
    lambda x: random.randint(1, 4) if pd.isna(x) else x
)

print("\nAfter creating new columns:")
df_restaurant_cleaned.head()


After creating new columns:


,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng,price_level,clean_name,ratings_count_rand,price_level_rand
0,1,19,PJ Fresh (224 Daniel Payne Drive),4.25,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,1.0,PJ Fresh,419.0,1.0
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,1.85,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330,NaN,J' ti`'z Smoothie-N-Coffee Bar,232.0,1.0
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),2.97,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640,1.0,Philly Fresh Cheesesteaks,200.0,1.0
3,4,17,Papa Murphy's (1580 Montgomery Highway),4.03,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614,1.0,Papa Murphy's,220.0,1.0
4,5,162,Nelson Brothers Cafe (17th St N),4.70,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700,NaN,Nelson Brothers Cafe,22.0,4.0


In [ ]:
df_restaurant_cleaned.to_csv("df_restaurant_cleaned.csv", index=False)

In [ ]:
import pandas as pd
import re

# Load your cleaned restaurant data (update the path if needed)
df_restaurant = pd.read_csv("df_restaurant_cleaned.csv")

# Extract simplified keywords from category field
def extract_keywords_from_category(text):
    if pd.isna(text):
        return []
    text = text.lower()
    text = re.sub(r'[^a-z0-9, ]', '', text)  # keep only alphanum, comma, space
    return [cat.strip() for cat in text.split(',') if cat.strip()]

# Apply the extraction
df_restaurant['category_keywords'] = df_restaurant['category'].apply(extract_keywords_from_category)

# Convert list to string format for saving
df_restaurant['category_keywords_str'] = df_restaurant['category_keywords'].apply(lambda x: ', '.join(x))

# Save relevant columns to CSV
df_keywords_out = df_restaurant[['id', 'name', 'category', 'category_keywords_str']]
df_keywords_out.to_csv("restaurant_category_keywords.csv", index=False)

print("✅ Saved to 'restaurant_category_keywords.csv'")

✅ Saved to 'restaurant_category_keywords.csv'


In [ ]:
import pandas as pd

# Load the menu and restaurant keyword files
df_menu_cleaned = pd.read_csv("df_menu_cleaned.csv")  # or your current menu df
df_rest_keywords = pd.read_csv("restaurant_category_keywords.csv")

# Rename 'id' in restaurant data to match 'restaurant_id' in menu
df_rest_keywords.rename(columns={'id': 'restaurant_id'}, inplace=True)

# Merge on restaurant_id
df_menu_joined = df_menu_cleaned.merge(df_rest_keywords[['restaurant_id', 'category_keywords_str']],
                                       on='restaurant_id', how='left')



In [ ]:
# Preview the result
df_menu_joined[['restaurant_id', 'name','clean_name', 'category_keywords_str']].head()

,restaurant_id,name,clean_name,category_keywords_str
0,1,Extra Large Meat Lovers,extra large meat lovers,"burgers, american, sandwiches"
1,1,Extra Large Supreme,extra large supreme,"burgers, american, sandwiches"
2,1,Extra Large Pepperoni,extra large pepperoni,"burgers, american, sandwiches"
3,1,Extra Large BBQ Chicken &amp; Bacon,extra large bbq chicken bacon,"burgers, american, sandwiches"
4,1,Extra Large 5 Cheese,extra large 5 cheese,"burgers, american, sandwiches"


In [ ]:
import re
import html

def clean_text(text):
    if pd.isna(text):
        return ""
    text = html.unescape(str(text)).lower()
    text = re.sub(r'[^a-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

if 'clean_category' not in df_menu_joined.columns:
    df_menu_joined['clean_category'] = df_menu_joined['category'].apply(clean_text)


# Step 1: define the same keyword → simplified category mapping
keyword_map = {
    'pizza': 'pizza',
    'burger': 'burgers',
    'sandwich': 'sandwiches',
    'chicken': 'chicken',
    'pasta': 'pasta',
    'noodle': 'noodles',
    'soup': 'soups',
    'salad': 'salads',
    'rice': 'rice dishes',
    'dessert': 'desserts',
    'cake': 'desserts',
    'steak': 'steak',
    'shrimp': 'seafood',
    'fish': 'seafood',
    'sushi': 'sushi',
    'drink': 'drinks',
    'beverage': 'drinks',
    'coffee': 'drinks',
    'tea': 'drinks',
    'side': 'sides',
    'breakfast': 'breakfast',
    'combo': 'combo meals'
}

# Step 2: function to match a keyword from a string
def match_from_text(text):
    if pd.isna(text):
        return None
    text = text.lower()
    for keyword, label in keyword_map.items():
        if keyword in text:
            return label
    return None

# Step 3: function to match from comma-separated category_keywords_str
def match_from_keywords(text):
    if pd.isna(text):
        return None
    for keyword in text.split(','):
        matched = match_from_text(keyword.strip())
        if matched:
            return matched
    return None

# Step 4: Apply logic with priority
def get_mapped_category(row):
    # Try the menu category first
    cat1 = match_from_text(row.get('clean_category', ''))
    if cat1:
        return cat1
    # Then try restaurant category keywords
    cat2 = match_from_keywords(row.get('category_keywords_str', ''))
    if cat2:
        return cat2
    return "misc"

# Apply to full joined DataFrame
df_menu_joined['mapped_category'] = df_menu_joined.apply(get_mapped_category, axis=1)

# Preview
df_menu_joined[['name', 'clean_name', 'clean_category', 'category_keywords_str', 'mapped_category']].head()

,name,clean_name,clean_category,category_keywords_str,mapped_category
0,Extra Large Meat Lovers,extra large meat lovers,extra large pizza,"burgers, american, sandwiches",pizza
1,Extra Large Supreme,extra large supreme,extra large pizza,"burgers, american, sandwiches",pizza
2,Extra Large Pepperoni,extra large pepperoni,extra large pizza,"burgers, american, sandwiches",pizza
3,Extra Large BBQ Chicken &amp; Bacon,extra large bbq chicken bacon,extra large pizza,"burgers, american, sandwiches",pizza
4,Extra Large 5 Cheese,extra large 5 cheese,extra large pizza,"burgers, american, sandwiches",pizza


In [ ]:
df_menu_joined.to_csv("df_menu_joined_maped.csv", index=False)

In [ ]:
df_menu_joined.head(10)

,restaurant_id,category,name,description,price,clean_name,price_clean,category_keywords_str,clean_category,mapped_category
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD,extra large meat lovers,15.99,"burgers, american, sandwiches",extra large pizza,pizza
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD,extra large supreme,15.99,"burgers, american, sandwiches",extra large pizza,pizza
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD,extra large pepperoni,14.99,"burgers, american, sandwiches",extra large pizza,pizza
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD,extra large bbq chicken bacon,15.99,"burgers, american, sandwiches",extra large pizza,pizza
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD,extra large 5 cheese,14.99,"burgers, american, sandwiches",extra large pizza,pizza
5,1,Extra Large Pizza,Extra Large Pepperoni Slice,Slice.,3.99 USD,extra large pepperoni slice,3.99,"burgers, american, sandwiches",extra large pizza,pizza
6,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon Slice,NaN,3.99 USD,extra large bbq chicken bacon slice,3.99,"burgers, american, sandwiches",extra large pizza,pizza
7,1,Extra Large Pizza,Extra Large Meat Lovers Slice,Slice.,3.99 USD,extra large meat lovers slice,3.99,"burgers, american, sandwiches",extra large pizza,pizza
8,1,Extra Large Pizza,Extra Large Supreme Slice,Slice.,3.99 USD,extra large supreme slice,3.99,"burgers, american, sandwiches",extra large pizza,pizza
9,1,Extra Large Pizza,Extra Large 5 Cheese Slice,Slice.,3.99 USD,extra large 5 cheese slice,3.99,"burgers, american, sandwiches",extra large pizza,pizza


# Match clean_name + mapped_category in restaurant as key to match with recipe clean title

In [ ]:
import pandas as pd
df_menu_joined = pd.read_csv("df_menu_joined_maped.csv")

In [ ]:
df_menu_joined['match_key'] = df_menu_joined['mapped_category'] + ' ' + df_menu_joined['clean_name']

In [ ]:
df_recipe_cleaned=pd.read_csv("df_recipe_cleaned.csv")
df_recipe_cleaned['match_key'] = df_recipe_cleaned['clean_title']

In [ ]:
!pip install rapidfuzz

In [ ]:
!pip install sentence_transformers

In [ ]:
import torch
print(torch.version.cuda)

12.4


In [ ]:
!pip install faiss-gpu-cu12

In [ ]:
!pip show faiss-gpu-cu12 | grep Version

Version: 1.10.0


In [ ]:
print("Number of GPUs:", faiss.get_num_gpus())

Number of GPUs: 0


In [ ]:
!pip install faiss-cpu

In [ ]:
!apt install libomp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


Cap the menu df to 250,000 rows

In [ ]:
import pandas as pd

# Create a new copy of your menu DataFrame
df_menu_copy = df_menu_joined.copy()

# Ensure that the 'restaurant_id' column exists. (Adjust column name if needed.)
if 'restaurant_id' not in df_menu_copy.columns:
    raise ValueError("Your df_menu_joined DataFrame must have a 'restaurant_id' column.")

# Step 1: Select one record per restaurant to ensure every restaurant is represented.
unique_items = df_menu_copy.groupby('restaurant_id', group_keys=False).apply(lambda x: x.head(1))

# Step 2: Identify the remaining menu items (those not already selected).
remaining_items = df_menu_copy.drop(unique_items.index)

# Step 3: Set your target cap and determine how many additional records you need.
target = 250000
remaining_needed = target - len(unique_items)

if remaining_needed > 0:
    # Randomly sample the additional records from the remaining items.
    additional_sample = remaining_items.sample(n=remaining_needed, random_state=42)
    # Concatenate the unique items with the additional sample.
    df_menu_cap = pd.concat([unique_items, additional_sample])
else:
    # If the unique items already exceed the target, sample down to the target.
    df_menu_cap = unique_items.sample(n=target, random_state=42)

# Optionally, shuffle the final DataFrame.
df_menu_cap = df_menu_cap.sample(frac=1, random_state=42).reset_index(drop=True)

print("Total records after capping:", len(df_menu_cap))
df_menu_cap.head()

<ipython-input-34-d02a06fb1a60>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_items = df_menu_copy.groupby('restaurant_id', group_keys=False).apply(lambda x: x.head(1))


Total records after capping: 250000


,restaurant_id,category,name,description,price,clean_name,price_clean,category_keywords_str,clean_category,mapped_category,match_key
0,38770,Picked for you,The Cheeseburger,"cheddar, red relish, pickled mustard seed mayo",19.0 USD,the cheeseburger,19.00,"burgers, american, sandwiches",picked for you,burgers,burgers the cheeseburger
1,19825,Spring Essentials,Coolhaus Bananas Foster Ice Cream (1 Pint),NaN,7.49 USD,coolhaus bananas foster ice cream 1 pint,7.49,"convenience, ice cream frozen yogurt, snacks,...",spring essentials,misc,misc coolhaus bananas foster ice cream 1 pint
2,3958,Picked for you,10 PC. Spicy Chicken Nuggets,100% white-meat chicken breaded and marinated ...,5.04 USD,10 pc spicy chicken nuggets,5.04,"american, burgers, fast food",picked for you,burgers,burgers 10 pc spicy chicken nuggets
3,59561,Delivery Cookie Dough,German Chocolate Cookie Dough Scoop*,our homemade cookie dough topped with a dollop...,4.55 USD,german chocolate cookie dough scoop,4.55,"american, desserts, bakery",delivery cookie dough,desserts,desserts german chocolate cookie dough scoop
4,47580,Desserts,Carrot Halwa,NaN,4.99 USD,carrot halwa,4.99,"indian, vegetarian friendly, desserts",desserts,desserts,desserts carrot halwa


In [ ]:
df_menu_cap.to_csv("df_menu_cap.csv", index=False)

In [ ]:
import faiss
import numpy as np
from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer, util
import torch, pandas as pd
from datetime import datetime
import concurrent.futures

# --- Prepare Composite Keys for Recipe and Menu Data ---

# For recipes, ensure clean_title is a string and create match_key.
df_recipe_cleaned['clean_title'] = df_recipe_cleaned['clean_title'].fillna("").astype(str)
df_recipe_cleaned['match_key'] = df_recipe_cleaned['clean_title'].str.lower().str.strip()

# Fill missing values and convert to string
df_menu_cap['mapped_category'] = df_menu_cap['mapped_category'].fillna("").astype(str)
df_menu_cap['clean_name'] = df_menu_cap['clean_name'].fillna("").astype(str)
df_menu_cap['description'] = df_menu_cap['description'].fillna("").astype(str)

# Replace "misc" (case insensitive) with an empty string.
df_menu_cap['mapped_category'] = df_menu_cap['mapped_category'].apply(
    lambda x: "" if x.strip().lower() == "misc" else x.strip()
)

# Now build the composite keys:
# For semantic matching (includes description):
df_menu_cap['match_key_sem'] = (
    df_menu_cap['mapped_category'].str.lower().str.strip() + " " +
    df_menu_cap['clean_name'].str.lower().str.strip() + " " +
    df_menu_cap['description'].str.lower().str.strip()
)

# For fuzzy matching (only uses mapped_category and clean_name):
df_menu_cap['match_key_fuzzy'] = (
    df_menu_cap['mapped_category'].str.lower().str.strip() + " " +
    df_menu_cap['clean_name'].str.lower().str.strip()
)


In [ ]:


# Ensure recipe IDs exist for linking later.
if 'recipe_id' not in df_recipe_cleaned.columns:
    df_recipe_cleaned = df_recipe_cleaned.reset_index().rename(columns={'index': 'recipe_id'})

# --- Load the SentenceTransformer Model ---
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Build the Recipe FAISS Index ---
recipe_keys = df_recipe_cleaned['match_key'].tolist()
recipe_embeddings = model.encode(recipe_keys, convert_to_tensor=True)
recipe_emb_np = recipe_embeddings.cpu().numpy().astype('float32')
faiss.normalize_L2(recipe_emb_np)
d = recipe_emb_np.shape[1]
index_cpu = faiss.IndexFlatIP(d)
index_cpu.add(recipe_emb_np)

# Try to transfer the index to GPU if available; otherwise, fall back to CPU.
try:
    gpu_count = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
    if gpu_count > 0:
        res = faiss.StandardGpuResources()
        gpu_index = faiss.index_cpu_to_gpu(res, 0, index_cpu)
        print("Using GPU index.")
    else:
        print("No GPUs detected by FAISS; using CPU index.")
        gpu_index = index_cpu
except Exception as e:
    print("Error transferring index to GPU:", e)
    print("Falling back to CPU index.")
    gpu_index = index_cpu

No GPUs detected by FAISS; using CPU index.


In [ ]:
# --- Set Matching Thresholds and Parameters ---
fuzzy_threshold = 70       # Minimum fuzzy score to consider a match
semantic_threshold = 0.7   # Minimum cosine similarity for a match
k_candidates = 5           # Retrieve top 5 candidates from FAISS for fuzzy refinement
batch_size = 128           # Adjust based on available memory

num_menu = len(df_menu_cap)
# We'll use the semantic key for encoding, and the fuzzy key for fuzzy matching.
# Prepare lists of keys and extra columns:
# (We won't precompute these embeddings now; we'll do it per batch.)
menu_sem_keys = df_menu_cap['match_key_sem'].tolist()
menu_fuzzy_keys = df_menu_cap['match_key_fuzzy'].tolist()
menu_names = df_menu_cap['name'].tolist()
restaurant_ids = df_menu_cap['restaurant_id'].tolist()
categories = df_menu_cap['mapped_category'].tolist()
prices = df_menu_cap['price_clean'].tolist()

# --- Batch Processing Function ---
def process_batch(batch_df, global_start):
    # For semantic matching, use match_key_sem.
    batch_sem_texts = batch_df['match_key_sem'].tolist()
    # For fuzzy matching, use match_key_fuzzy.
    batch_fuzzy_texts = batch_df['match_key_fuzzy'].tolist()
    batch_names = batch_df['name'].tolist()
    batch_rest_ids = batch_df['restaurant_id'].tolist()
    batch_categories = batch_df['mapped_category'].tolist()
    batch_prices = batch_df['price_clean'].tolist()

    # Encode the semantic key texts for this batch.
    batch_embeddings = model.encode(batch_sem_texts, convert_to_tensor=True)
    batch_emb_np = batch_embeddings.cpu().numpy().astype('float32')
    faiss.normalize_L2(batch_emb_np)

    # Retrieve top-k candidates from the recipe FAISS index.
    D, I = gpu_index.search(batch_emb_np, k=k_candidates)
    batch_results = []
    for j, sem_text in enumerate(batch_sem_texts):
        semantic_score = D[j][0]
        best_idx = I[j][0]
        semantic_recipe = recipe_keys[best_idx]
        # For fuzzy matching, use the fuzzy key.
        fuzzy_text = batch_fuzzy_texts[j]
        if semantic_score < semantic_threshold:
            shared_key = None
            fuzzy_match = None
            fuzzy_score = 0
        else:
            # Refine fuzzy matching on the top-k candidates.
            candidates = [recipe_keys[idx] for idx in I[j]]
            fuzzy_match = process.extractOne(fuzzy_text, candidates, scorer=fuzz.token_sort_ratio)
            fuzzy_score = fuzzy_match[1] if fuzzy_match else 0
            if fuzzy_score >= fuzzy_threshold:
                candidate_idx = I[j][candidates.index(fuzzy_match[0])]
                shared_key = df_recipe_cleaned.iloc[candidate_idx]['recipe_id']
                semantic_recipe = recipe_keys[candidate_idx]
                semantic_score = D[j][list(candidates).index(fuzzy_match[0])]
            else:
                shared_key = None

        batch_results.append({
            'menu_name': batch_names[j],
            'menu_sem_key': sem_text,
            'menu_fuzzy_key': fuzzy_text,
            'restaurant_id': batch_rest_ids[j],
            'category': batch_categories[j],
            'price_clean': batch_prices[j],
            'fuzzy_recipe': fuzzy_match[0] if fuzzy_match else None,
            'fuzzy_score': fuzzy_score if fuzzy_match else None,
            'semantic_recipe': semantic_recipe,
            'semantic_score': round(float(semantic_score), 4),
            'shared_key': shared_key
        })
    print(f"{datetime.now()}: Processed up to record {global_start + len(batch_df)} / {num_menu} menu items")
    return batch_results

# --- Prepare DataFrame Slices for Batching with Global Start Indices ---
batch_args_list = []
for start in range(0, num_menu, batch_size):
    end = min(start + batch_size, num_menu)
    batch_df = df_menu_cap.iloc[start:end].copy().reset_index(drop=True)
    batch_args_list.append((batch_df, start))

# --- Use ThreadPoolExecutor for Parallel Batch Processing ---
results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    all_batches = list(executor.map(lambda args: process_batch(*args), batch_args_list))
for batch in all_batches:
    results.extend(batch)

df_matches_combined = pd.DataFrame(results)
df_matches_combined.head()

2025-04-05 08:17:49.291866: Processed up to record 768 / 250000 menu items
2025-04-05 08:17:50.556537: Processed up to record 128 / 250000 menu items
2025-04-05 08:17:50.697422: Processed up to record 1408 / 250000 menu items
2025-04-05 08:17:50.701800: Processed up to record 1152 / 250000 menu items
2025-04-05 08:17:51.148972: Processed up to record 1024 / 250000 menu items
2025-04-05 08:17:51.787527: Processed up to record 384 / 250000 menu items
2025-04-05 08:17:51.850965: Processed up to record 512 / 250000 menu items
2025-04-05 08:17:51.914843: Processed up to record 256 / 250000 menu items
2025-04-05 08:17:51.949592: Processed up to record 896 / 250000 menu items
2025-04-05 08:17:51.963178: Processed up to record 1280 / 250000 menu items
2025-04-05 08:17:51.991112: Processed up to record 640 / 250000 menu items
2025-04-05 08:17:52.031764: Processed up to record 1536 / 250000 menu items
2025-04-05 08:17:58.029728: Processed up to record 1664 / 250000 menu items
2025-04-05 08:18:01

,menu_name,menu_sem_key,menu_fuzzy_key,restaurant_id,category,price_clean,fuzzy_recipe,fuzzy_score,semantic_recipe,semantic_score,shared_key
0,The Cheeseburger,"burgers the cheeseburger cheddar, red relish, ...",burgers the cheeseburger,38770,burgers,19.00,hot mustard and chicken cheese burgers,54.838710,cheddar turkey burgers with roasted red pepper...,0.7906,NaN
1,Coolhaus Bananas Foster Ice Cream (1 Pint),coolhaus bananas foster ice cream 1 pint,coolhaus bananas foster ice cream 1 pint,19825,,7.49,bananas foster ice cream pie,79.411765,bananas foster ice cream pie,0.7371,1728882.0
2,10 PC. Spicy Chicken Nuggets,burgers 10 pc spicy chicken nuggets 100% white...,burgers 10 pc spicy chicken nuggets,3958,burgers,5.04,hot spicy chicken burgers,76.666667,hot spicy chicken burgers,0.7642,1094493.0
3,German Chocolate Cookie Dough Scoop*,desserts german chocolate cookie dough scoop o...,desserts german chocolate cookie dough scoop,59561,desserts,4.55,german chocolate cake cookies,65.753425,german chocolate brownie cookies,0.7686,NaN
4,Carrot Halwa,desserts carrot halwa,desserts carrot halwa,47580,desserts,4.99,carrot halwa indian carrot dessert,72.727273,carrot halwa indian carrot dessert,0.9441,1563385.0


In [ ]:
df_recipe_cleaned.head()

,recipe_id,Unnamed: 0,title,ingredients,directions,link,source,NER,clean_title,match_key
0,0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","['In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...",nobake nut cookies,nobake nut cookies
1,1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...",['Place chipped beef on bottom of baking dish....,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"['beef', 'chicken breasts', 'cream of mushroom...",jewell balls chicken,jewell balls chicken
2,2,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","['Boil and debone chicken.', 'Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"['chicken', 'chicken gravy', 'cream of mushroo...",chicken funny,chicken funny
3,3,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...",['Combine first four ingredients and press in ...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"['peanut butter', 'graham cracker crumbs', 'bu...",reeses cupscandy,reeses cupscandy
4,4,5,Cheeseburger Potato Soup,"['6 baking potatoes', '1 lb. of extra lean gro...",['Wash potatoes; prick several times with a fo...,www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"['baking potatoes', 'extra lean ground beef', ...",cheeseburger potato soup,cheeseburger potato soup


In [ ]:
df_restaurant_cleaned = pd.read_csv("df_restaurant_cleaned.csv")
df_restaurant_cleaned.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng,price_level,clean_name,ratings_count_rand,price_level_rand
0,1,19,PJ Fresh (224 Daniel Payne Drive),4.25,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,1.0,PJ Fresh,419.0,1.0
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,1.85,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330,NaN,J' ti`'z Smoothie-N-Coffee Bar,232.0,1.0
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),2.97,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640,1.0,Philly Fresh Cheesesteaks,200.0,1.0
3,4,17,Papa Murphy's (1580 Montgomery Highway),4.03,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614,1.0,Papa Murphy's,220.0,1.0
4,5,162,Nelson Brothers Cafe (17th St N),4.70,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700,NaN,Nelson Brothers Cafe,22.0,4.0


In [ ]:
df_matches_combined.to_csv("df_menu_matched_version1.csv", index=False)
df_recipe_cleaned.to_csv("df_recipe_cleaned_version1.csv", index=False)
df_restaurant_cleaned.to_csv("df_restaurant_cleaned_version1.csv", index=False)

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
